In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("app").master("local[2]").getOrCreate()

In [0]:
schema = StructType([
    StructField("product_id",IntegerType(),False),
    StructField("store1",IntegerType(),True),
    StructField("store2",IntegerType(),True),
    StructField("store3",IntegerType(),True)
])
data = [
( 0          , 95     , 100    , 105  )  ,
( 1          , 70     , None   , 80   )
]
products = spark.createDataFrame(data,schema)
products.show()

+----------+------+------+------+
|product_id|store1|store2|store3|
+----------+------+------+------+
|         0|    95|   100|   105|
|         1|    70|  null|    80|
+----------+------+------+------+



In [0]:
# Write a solution to rearrange the Products table so that each row has (product_id, store, price). 
# If a product is not available in a store, do not include a row with that product_id and store combination in the result table.
# Return the result table in any order.

products.unpivot("product_id",["store1","store2","store3"],"store","price").filter(col("price").isNotNull()).show()

+----------+------+-----+
|product_id| store|price|
+----------+------+-----+
|         0|store1|   95|
|         0|store2|  100|
|         0|store3|  105|
|         1|store1|   70|
|         1|store3|   80|
+----------+------+-----+



In [0]:
products.createOrReplaceTempView("p")
spark.sql("""select product_id,'store1' as store, store1 as price from p where store1 is not null 
            union  
            select product_id,'store2' as store, store2 as price from p where store2 is not null
            union
            select product_id,'store3' as store, store3 as price from p where store3 is not null
            """).show()


+----------+------+-----+
|product_id| store|price|
+----------+------+-----+
|         0|store1|   95|
|         1|store1|   70|
|         0|store2|  100|
|         0|store3|  105|
|         1|store3|   80|
+----------+------+-----+



In [0]:
spark.stop()